In [4]:
# preliminary stuff - get demo BIDS dataset
!datalad install ///workshops/nipype-2017/ds000114
!datalad remove ds000114/derivatives/*
!datalad get -r ds000114
# also double up import to avoid ugly warnings printing during slideshow
from bids.layout import BIDSLayout
import os.path
import shutil
# house keeping
if os.path.exists("output"):
    shutil.rmtree("output", ignore_errors=True)

drop(error): /home/jc01/python/notebook_bids_OSD2018/ds000114/derivatives/* (file) [not found]
[INFO   ] Installing Dataset(/home/jc01/python/notebook_bids_OSD2018) to get /home/jc01/python/notebook_bids_OSD2018/ds000114 recursively 


In [5]:
# and demo dicoms for dcm2bids
!datalad install -g ///dicoms/dartmouth-phantoms/bids_test5-20170120

In [6]:
import json
def writeconfig(config, filename="config.json"):
    with open(filename, "w") as fp:
        json.dump(config, fp, indent=4)

config = dict(descriptions=[])
config["descriptions"].append({"dataType": "anat", 
                               "suffix": "T1w",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": 
                                       "anat_T1w_acq-MPRAGE_run"
                                   }
                                   }
                               })
# we need a task for valid BIDS - "rest" is handy because it means we are exempt from
# needing the events.tsv file (phantoms are at rest by definition, right?)
config["descriptions"].append({"dataType": "func", 
                               "suffix": "bold",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "bold"
                                   }
                                   },
                               "customHeader": {"TaskName": "rest"}
                               })
# unfortunately no real way to distinguish the fieldmap magnitude images without
# matching on the echo times
echo1 = 0.00492
echo2 = 0.00738
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude1",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoTime": echo1
                                   }
                                   }
                               })
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "magnitude2",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType":
                                       ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
                                       "EchoNumber": 2,
                                       "EchoTime": echo2
                                   }
                                   }
                               })
# need to intervene here to set EchoTime1 and EchoTime2 since it's a phase difference
# image (see https://github.com/rordenlab/dcm2niix/issues/139)
config["descriptions"].append({"dataType": "fmap", 
                               "suffix": "phasediff",
                               "criteria": {
                                   "in": {
                                       "PulseSequenceDetails": "field_mapping"
                                   },
                                   "equal": {
                                       "ImageType": ["ORIGINAL", "PRIMARY", "P", "ND"]
                                   }
                                   },
                               "customHeader": {
                                   "EchoTime1": echo1,
                                   "EchoTime2": echo2
                               }
                               })
writeconfig(config, filename="config_complete.json")

# <center>Brain Imaging Data Structure (BIDS): A standard format for neuroscience data</center>
<p><center>Johan Carlin</center></p>
<p><center><a href="https://github.com/jooh/notebook_bids_OSD2018">github.com/jooh/notebook_bids_OSD2018</a></center></p>
<p><center><a href="https://mybinder.org/v2/gh/jooh/notebook_bids_OSD2018/master?filepath=os2018_bids.ipynb"><img src="https://mybinder.org/badge_logo.svg"></center></p>
<p><center><img src="MRC_CBU_Cambridge_colour_web_A5.png"></center></p>

## In BIDS 1.1.1
* MRI - anatomical, functional, diffusion, field maps…
* MEG
* Physiological recordings
* Behavioural data

## Draft extensions
* Model specification
* Derivatives - structural, functional, resting state, diffusion…
* New modalities - EEG, PET, ASL, Eye tracking, intracranial EEG

# MRI example

In [7]:
# basic directory structure
!tree -CL 2 ds000114

ds000114
├── dataset_description.json
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── anat
│   ├── dwi
│   └── func
├── sub-02
│   ├── anat
│   ├── dwi
│   └── func
├── task-fingerfootlips_bold.json
├── task-fingerfootlips_events.tsv
└── task-linebisection_bold.json

8 directories, 6 files


In [8]:
# and inside one func folder
subdir = "ds000114/sub-01/func"
!ls -1 {subdir}
# and an events file
#import pandas as pd
#pd.read_csv(
#    f"{subdir}/sub-01_task-linebisection_events.tsv",delimiter="\t")

sub-01_task-fingerfootlips_bold.nii.gz
sub-01_task-linebisection_bold.nii.gz
sub-01_task-linebisection_events.tsv


# Why BIDS?

**Users** get easy access to new analysis tools, e.g.
`fmriprep ds000114 outdir sub-01 -w workdir`

In [9]:
# Developers:
from bids.layout import BIDSLayout
layout = BIDSLayout("ds000114")
subjects = layout.get_subjects()
print(subjects)
#layout.get(subject=subjects[0], type="T1w", return_type="file")

/home/jc01/.conda/envs/notebook_bids_osd2018/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


['01', '02']


# Dcm2Bids
Fully automated conversion from dicom to BIDS-compliant output data.
* Converts dicoms to nifti (using dcm2niix), optionally anonymises structurals
* Moves niftis and header sidecars to the correct BIDS-format locations
* Adds custom fields to header sidecars as needed for BIDS compliance
* Initialises all required BIDS project files with sensible defaults

# Input dicoms

In [10]:
!tree -CL 1 bids_test5-20170120/phantom-1/

bids_test5-20170120/phantom-1/
├── 001-anat-scout_run+
├── 002-anat-scout_run+_MPR_sag
├── 003-anat-scout_run+_MPR_cor
├── 004-anat-scout_run+_MPR_tra
├── 005-anat_T1w_acq-MPRAGE_run+
├── 006-fmap_acq-2.4mm
├── 007-fmap_acq-2.4mm
└── 009-func_run+_task-rest_acq-2.4mm64sl1000tr32te600dyn

8 directories, 0 files


# Minimal configuration for a first test run
dcm2bids uses a configuration JSON file to map from dicom series to BIDS output files

In [11]:
# you could just type it out but we we will extend the dict below
config = dict(descriptions=[])
writeconfig(config)
!cat config.json

{
    "descriptions": []
}

In [12]:
# First test run of dcm2bids
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004_i00001.json' satisfies no description - skipping
'bids_t

In [13]:
# Unsurprisingly there's not much in the output dir
# but NB the tmp_dcm2bids dir!
!tree -C output

output
├── dcm2bids.log
└── tmp_dcm2bids
    └── sub-01
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series001.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series001.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00001.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00001.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00002.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00002.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00003.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00003.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00004.json
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00004.nii.gz
        ├── bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00001.json
        ├── b

In [14]:
# so the trick is to extend the config json to unambiguously identify our target
# acquisitions. Let's start with the t1. What's in the sidecar?
!cat output/tmp_dcm2bids/sub-01/*T1w*.json

{
	"Modality": "MR",
	"MagneticFieldStrength": 3,
	"ImagingFrequency": 123.253,
	"Manufacturer": "Siemens",
	"ManufacturersModelName": "Prisma",
	"InstitutionName": "Dartmouth College - PBS",
	"InstitutionalDepartmentName": "Department",
	"InstitutionAddress": "Maynard 3,Hanover,NH,US,03755",
	"DeviceSerialNumber": "66112",
	"StationName": "AWP66112",
	"BodyPartExamined": "BRAIN",
	"PatientPosition": "HFS",
	"ProcedureStepDescription": "Heatherton_Stanger^1004_ontrack",
	"SoftwareVersions": "syngo MR E11",
	"MRAcquisitionType": "3D",
	"SeriesDescription": "anat_T1w_acq-MPRAGE_run+",
	"ProtocolName": "anat_T1w_acq-MPRAGE_run+",
	"ScanningSequence": "GR\\IR",
	"SequenceVariant": "SK\\SP\\MP",
	"ScanOptions": "IR",
	"SequenceName": "*tfl3d1_16ns",
	"ImageType": ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"],
	"SeriesNumber": 5,
	"AcquisitionTime": "08:44:38.722500",
	"AcquisitionNumber": 1,
	"ReconstructionMethod": "Éü",
	"SliceThickness": 0.9,
	"SAR": 0.0381398,
	"EchoTime": 0.00232,
	"Rep

In [15]:
# so our first description might be
config["descriptions"] = []
config["descriptions"].append({"dataType": "anat", 
                               "suffix": "T1w",
                               "criteria": {
                                   "in": {
                                       "SeriesDescription": 
                                       "anat_T1w_acq-MPRAGE_run"
                                   }
                                   }
                               })
# save and print the config again
writeconfig(config)
!cat config.json

{
    "descriptions": [
        {
            "dataType": "anat",
            "suffix": "T1w",
            "criteria": {
                "in": {
                    "SeriesDescription": "anat_T1w_acq-MPRAGE_run"
                }
            }
        }
    ]
}

In [16]:
# take 2 - do we recognise the t1?
!dcm2bids -d bids_test5-20170120 -p 01 -c config.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004_i00001.json' satisfies no description - skipping
'bids_t

In [17]:
# now the output dir looks more interesting
!tree -C -I tmp_dcm2bids output

output
├── CHANGES
├── dataset_description.json
├── dcm2bids.log
├── derivatives
├── participants.tsv
├── README
└── sub-01
    └── anat
        ├── sub-01_T1w.json
        └── sub-01_T1w.nii.gz

3 directories, 7 files


In [18]:
# some of the required BIDS project root files have been initialised, e.g.
!cat output/dataset_description.json

{
    "Name": "",
    "BIDSVersion": "1.1.0",
    "License": "",
    "Authors": [
        ""
    ],
    "Acknowledgments": "",
    "HowToAcknowledge": "",
    "Funding": "",
    "ReferencesAndLinks": [
        ""
    ],
    "DatasetDOI": ""
}

# Scaling up to a complete dcm2bids conversion

In [19]:
# here's one I made earlier...
!cat config_complete.json

{
    "descriptions": [
        {
            "dataType": "anat",
            "suffix": "T1w",
            "criteria": {
                "in": {
                    "SeriesDescription": "anat_T1w_acq-MPRAGE_run"
                }
            }
        },
        {
            "dataType": "func",
            "suffix": "bold",
            "criteria": {
                "in": {
                    "PulseSequenceDetails": "bold"
                }
            },
            "customHeader": {
                "TaskName": "rest"
            }
        },
        {
            "dataType": "fmap",
            "suffix": "magnitude1",
            "criteria": {
                "in": {
                    "PulseSequenceDetails": "field_mapping"
                },
                "equal": {
                    "ImageType": [
                        "ORIGINAL",
                        "PRIMARY",
                        "M",
                        "ND",
                        "NORM"
                   

In [20]:
# take 3 - do we now recognise all the series we want to convert?
shutil.rmtree("output", ignore_errors=True)
!dcm2bids -d bids_test5-20170120 -p 01 -c config_complete.json -o output


'bids_test5-20170120_anat-scout_run+_20170120084340_series001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series002_i00004.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00001.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00002.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series003_i00003.json' satisfies no description - skipping
'bids_test5-20170120_anat-scout_run+_20170120084340_series004_i00001.json' satisfies no description - skipping
'bids_t

In [21]:
# the output directory looks fairly complete
!tree -C -I tmp_dcm2bids output

output
├── CHANGES
├── dataset_description.json
├── dcm2bids.log
├── derivatives
├── participants.tsv
├── README
└── sub-01
    ├── anat
    │   ├── sub-01_T1w.json
    │   └── sub-01_T1w.nii.gz
    ├── fmap
    │   ├── sub-01_magnitude1.json
    │   ├── sub-01_magnitude1.nii.gz
    │   ├── sub-01_magnitude2.json
    │   └── sub-01_magnitude2.nii.gz
    └── func
        ├── sub-01_task-rest_bold.json
        └── sub-01_task-rest_bold.nii.gz

5 directories, 13 files


# Checking conversions with bids-validator
Also available as a handy web app

In [22]:
# NB you would still need to add some more info manually in e.g. study_description.json
!bids-validator output

	1: [ERR] Empty files not allowed. (code: 99 - EMPTY_FILE)
		./README

	Please visit https://neurostars.org/search?q=EMPTY_FILE for existing conversations about this issue.

	1: [WARN] The Authors field of dataset_description.json should contain an array of fields - with one author per field. This was triggered based on the presence of only one author field. Please ignore if all contributors are already properly listed. (code: 102 - TOO_FEW_AUTHORS)

	Please visit https://neurostars.org/search?q=TOO_FEW_AUTHORS for existing conversations about this issue.


        Summary:                Available Tasks:        Available Modalities: 
        12 Files, 18.4MB        rest                    T1w                   
        1 - Subject                                     magnitude1            
        1 - Session                                     magnitude2            
                                                        bold                  


	If you have any questions, please post

# Resources

* [The BIDS starter kit](https://github.com/bids-standard/bids-starter-kit) - wiki with links to various resources
* [The official BIDS website](http://bids.neuroimaging.io/) - the official BIDS specification is surprisingly readable
* [Michael Notter's Nipype tutorial](https://github.com/miykael/nipype_tutorial) - great interactive tutorials on working with BIDS data in Python
* [BIDS-Validator web app](https://bids-standard.github.io/bids-validator/) - convenient way to check BIDS conversions
* [dcm2bids (jooh fork)](https://github.com/jooh/Dcm2Bids) - run the conversion code presented here